In [1]:
# Initialization
#x = np.array([-np.pi/4, -np.pi/2, np.pi/4, 0, 0, 0]) # Initial robot pose
x_target = np.array([-np.pi/4, -np.pi/2, np.pi/4]) # Target in joint space

param.damping = 20.0 # Viscous friction

#kP = 400.0 # Stiffness gain
#kV = 10.0 # Damping gain

KP = np.diag([4E2, 4E2, 4E2]) # Joint space stiffness matrix
KV = np.diag([1E1, 1E1, 1E1]) # Joint space damping matrix

# Control loop definition
def control(x, param):
    # Torques for gravity compensation
    #ug = inverse_dynamics(np.append(x[:param.nbVarX], np.zeros(param.nbVarX)), np.zeros(param.nbVarX), param)
    ug = inverse_dynamics(x, np.zeros(param.nbVarX), param) # Torques for gravity and Coriolis force compensation # 科里奥利
    #u = kP * (x_target - x[:param.nbVarX]) - kV * x[param.nbVarX:] + ug # Impedance controller in joint space
    u = KP @ (x_target - x[:param.nbVarX]) - KV @ x[param.nbVarX:] + ug # Impedance controller in joint space
    return u

NameError: name 'np' is not defined

In [2]:
import numpy as np
x = np.array([-np.pi/4, -np.pi/2, np.pi/4, 0, 0, 0]) # Initial robot pose

dist_target = 100.0 # Targeted distance to maintain
f_target = np.array([-200.0, -400.0, 0]) # SDF location in task space # Signed Distance Function (SDF)
sdf_disc_radius = 80.0 # Disc radius # 圆盘
sdf_box_size = np.array([160.0, 100.0]) # Box width and height
sdf_box_offset = np.array([60.0, -60.0]) # Box position wrt the disc # wrt: with regard to
sdf_smoothing_ratio = 10.0 # Smoothing factor for softmax composition of SDF shapes # softmax:柔性最大值传输函数

KP = np.diag([4E-2, 0E-2, 0E3]) # Task space stiffness (position and orientation)
KP0 = np.copy(KP)
KV = np.diag([1E-3, 1E-3, 1E1]) # Task space damping (position and orientation)

def control(x, param):
    global KP
    ug = inverse_dynamics(x, np.zeros(param.nbVarX), param) # Torques for gravity and Coriolis force compensation
    f = fkin(x[:param.nbVarX], param) # Forward kinematics
    J = Jkin(x[:param.nbVarX], param) # Corresponding Jacobian matrix
    df = J @ x[param.nbVarX:] # End-effector velocity
    dist, grad = sdf(f) # Signed distance function and corresponding gradient
    R = np.array([[grad[0], -grad[1]], [grad[1], grad[0]]]) # Local coordinate system (rotation matrix)
    KP[:2,:2] = R @ KP0[:2,:2] @ R.T # Adapt stiffness to local coordinate system
    grad[:2] = grad[:2] * (dist - dist_target) # Residual vector
    u = J.T @ (KP @ grad - KV @ df) + ug # Impedance controller in task space
    return u

In [7]:
import numpy as np

n1 = np.array([1,2,3])
n2 = n1

n3 = n1.copy()
n4 = np.copy(n1)
n1[0] = 0
print(n1)
print(n2)
print(n3)
print(n4)
print(id(n1))
print(id(n2))
print(id(n3))
print(id(n4))

[0 2 3]
[0 2 3]
[1 2 3]
[1 2 3]
140243114410128
140243114410128
140243114411760
140243114411088
